# Experiment 02

- [ ] mask entries in larger dataset in long-format
- [ ] mask peptides based on their frequency in samples (probability of being observed)
- [ ] create training data set without masked values for each model
    - Denoising AE
    - FNN based on embeddings (Collaborative Filtering)
    - VAE
- [ ] restrict to only a training data split of consective data: Increase number of samples.
    - focus on best reconstruction performance
    - mean comparison

### Collaborative Filtering model
- Cannot accomodate iid assumption of statistical test in current setup for embedding vectors.
    - if pretrained model should be applied to an new batch of replicates (with a certain condition)
      one would need to find a way to initialize the sample embeddings without fine-tuning the model

In [ ]:
from pprint import pprint
from src.nb_imports import *


import vaep.io_images
import seaborn

import numpy.testing as npt # fastcore.test functionality

from pathlib import Path
from src import metadata


import logging
from src.logging import setup_logger

logger = setup_logger(logger=logging.getLogger('vaep'))
logger.info("Experiment 02")

figures = {}  # collection of ax or figures

ADD_TENSORBOARD = False

In [ ]:
# None takes all
N_SAMPLES = 500

## Raw data

In [ ]:
# only some sample have many missings

FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / \
    'df_intensities_N_07813_M01000'  # all
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / \
    'df_intensities_N_07637_M01000'  # 60%
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / \
    'df_intensities_N_07285_M01000'  # 90%

In [ ]:
analysis = AnalyzePeptides(fname=FN_PEPTIDE_INTENSITIES, nrows=None)
analysis.df.columns.name = 'peptide'
analysis.log_transform(np.log2)
analysis

In [ ]:
# some date are not possible in the indices
rename_indices_w_wrong_dates = {'20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_03': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_03',
                                '20180230_QE10_nLC0_MR_QC_MNT_Hela_12': '20180330_QE10_nLC0_MR_QC_MNT_Hela_12',
                                '20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_01': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_01',
                                '20180230_QE10_nLC0_MR_QC_MNT_Hela_11': '20180330_QE10_nLC0_MR_QC_MNT_Hela_11',
                                '20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_02': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_02'}
analysis.df.rename(index=rename_indices_w_wrong_dates, inplace=True)

### Select N consecutive samples

In [ ]:
# sort index
analysis.df.sort_index(inplace=True)
analysis.df_all = analysis.df

In [ ]:
import random
N_SAMPLES = min(len(analysis.df), N_SAMPLES) if N_SAMPLES else len(analysis.df)

random.seed(42)


def get_consecutive_data_indices(df, n_samples=N_SAMPLES):
    index = df.sort_index().index
    start_sample = len(index) - n_samples
    start_sample = random.randint(0, start_sample)
    return df.loc[index[start_sample:start_sample+n_samples]]


_attr_name = f'df_{N_SAMPLES}'
setattr(analysis, _attr_name, get_consecutive_data_indices(analysis.df_all))
print("Training data stored under:", _attr_name)
analysis.df = getattr(analysis, _attr_name)
analysis.df

In [ ]:
assert not analysis.df._is_view

- biological stock differences in PCA plot. Show differences in models. Only see biological variance

In [ ]:
d_meta = metadata.get_metadata_from_filenames(analysis.df.index)
analysis.df_meta = pd.DataFrame.from_dict(
    d_meta, orient='index')
analysis.df_meta

Use to find date parsing errors, used for renaming above.

In [ ]:
# invalid_dates = pd.to_datetime(analysis.df_meta.date, errors='coerce').isna()
# display(analysis.df_meta.loc[invalid_dates])
# {i : i for i in analysis.df_meta.loc[invalid_dates].index} # to rename

In [ ]:
analysis.df_meta.describe()

See rare instrument types (potential labeling errors)

In [ ]:
N_MIN_INSTRUMENT = 10
ms_instruments = analysis.df_meta.ms_instrument.value_counts()
ms_instruments = ms_instruments[ms_instruments > N_MIN_INSTRUMENT].index
mask = ~analysis.df_meta.ms_instrument.isin(ms_instruments)
analysis.df_meta.loc[mask]

### PCA plot of raw data

In [ ]:
from collections import namedtuple
import itertools
from sklearn.impute import SimpleImputer
X = SimpleImputer().fit_transform(analysis.df)
X = vaep.pandas._add_indices(X, analysis.df)
assert X.isna().sum().sum() == 0

pca = analyzers.run_pca(X)
cols = list(pca.columns)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(
    15, 20), constrained_layout=True)

Dim = namedtuple('DimensionsData', 'N M')
analysis.dim = Dim(*analysis.df.shape)

fig.suptitle(
    f'First two Principal Components of {analysis.dim.M} most abundant peptides \n for {analysis.dim.N} samples', fontsize=30)


# by instrument
ax = axes[0]
pca['ms_instrument'] = analysis.df_meta['ms_instrument'].astype('category')
# for name, group in pca.groupby('ms_instrument'):
#     ax.scatter(x=group[cols[0]], y=group[cols[1]], label=name)
seaborn.scatterplot(x=pca[cols[0]], y=pca[cols[1]],
                    hue=pca['ms_instrument'], ax=ax, palette='deep')
ax.set_title('by category', fontsize=18)
ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))

# by dates
ax = axes[1]
ax.set_title('by date', fontsize=18)
path_collection = analyzers.scatter_plot_w_dates(
    ax, pca, dates=analysis.df_meta.date, errors='raise')
path_collection = analyzers.add_date_colorbar(path_collection, fig)

In [ ]:
vaep.io_images._savefig(fig, config.FIGUREFOLDER /
                        f'pca_plot_raw_data_{analysis.fname_stub}')

Scatter plots need to become interactive.

## Long format

- Data in long format: (peptide, sample_id, intensity)
- no missing values kept
- 

In [ ]:
def get_long_format(self, colname_values='intensity', inplace=False):
    df = self.df
    df_long = df.unstack().dropna().to_frame(colname_values)
    df_long = df_long.reset_index('Sample ID')
    if inplace:
        self.df_long = df_long
        return
    return df_long


get_long_format(analysis, inplace=True)
analysis.df_long.head()

In [ ]:
assert analysis.df_long.isna().sum().sum(
) == 0, "There are still missing values in the long format."

In [ ]:
def get_wide_format(self, columns='Sample ID', name_values='intensity', inplace=False):
    df_wide = self.df_long.pivot(columns=columns, values=name_values)
    df_wide = df_wide.T
    if inplace:
        self.df_wide = df_wide
        return
    return df_wide


get_wide_format(analysis, inplace=True)
analysis.df_wide.head()

In [ ]:
assert analysis.df_wide.isna().sum().sum(
) > 0, "There are no missing values left in the wide format"

### Sampling peptides by their frequency (important for later)

- higher count, higher probability to be sampled into training data
- missing peptides are sampled both into training as well as into validation dataset
- everything not in training data is validation data

In [ ]:
# freq_per_peptide = analysis.df.unstack().to_frame('intensity').reset_index(1, drop=True)
freq_per_peptide = analysis.df_long['intensity']
freq_per_peptide = freq_per_peptide.notna().groupby(level=0).sum()

In [ ]:
# df_long = analysis.df.unstack().to_frame('intensity').reset_index(1)
analysis.df_train = analysis.df_long.groupby(
    by='Sample ID').sample(frac=0.95, weights=freq_per_peptide, random_state=42)
analysis.df_train = analysis.df_train.reset_index().set_index([
    'Sample ID', 'peptide'])
analysis.df_train

## MultiIndex 

- use mulitindex for obtaining validation split

In [ ]:
analysis.df_long = analysis.df_long.reset_index(
).set_index(['Sample ID', 'peptide'])
analysis.df_long.head()

In [ ]:
analysis.indices_valid = analysis.df_long.index.difference(
    analysis.df_train.index)
analysis.df_valid = analysis.df_long.loc[analysis.indices_valid]

In [ ]:
assert len(analysis.df_long) == len(analysis.df_train) + len(analysis.df_valid)

## Setup DL

In [ ]:
import vaep.model as vaep_model
from vaep.cmd import get_args

BATCH_SIZE, EPOCHS = 128, 30
args = get_args(batch_size=BATCH_SIZE, epochs=EPOCHS,
                no_cuda=False)  # data transfer to GPU seems slow
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

# torch.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")
device

print(args, device, sep='\n')

Fastai default device for computation

In [ ]:
import fastai.torch_core
# # device = torch.device('cpu')
# # fastai.torch_core.defaults.device = torch.device('cpu')
# device = fastai.torch_core.defaults.device
# device
torch.cuda.is_available()

In [ ]:
fastai.torch_core.defaults

## Collaboritive filtering model

In [ ]:
from fastai.collab import CollabDataLoaders, MSELossFlat, Learner

analysis.collab = Analysis()
collab = analysis.collab
collab.columns = 'peptide,Sample ID,intensity'.split(',')

Create data view for collaborative filtering

- currently a bit hacky as the splitter does not support predefinded indices (create custum subclass providing splits to internal methods?)

- Use the [`CollabDataLoaders`](https://docs.fast.ai/collab.html#CollabDataLoaders)  similar to the [`TabularDataLoaders`](https://docs.fast.ai/tabular.data.html#TabularDataLoaders).
- Use the [`IndexSplitter`](https://docs.fast.ai/data.transforms.html#IndexSplitter) and provide splits to whatever is used in `CollabDataLoaders`


In [ ]:
collab.df_train = analysis.df_train.reset_index()
collab.df_valid = analysis.df_valid.reset_index()
collab.df_train.head()

In [ ]:
collab.df_valid.head()

In [ ]:
assert (collab.df_train.intensity.isna().sum(),
        collab.df_valid.intensity.isna().sum()) == (0, 0), "Remove missing values."

Hacky part uses training data `Datasets` from dataloaders to recreate a custom `DataLoaders` instance

In [ ]:
collab.dl_train = CollabDataLoaders.from_df(
    collab.df_train, valid_pct=0.0, user_name='Sample ID', item_name='peptide', rating_name='intensity', bs=args.batch_size, device=device)
collab.dl_valid = CollabDataLoaders.from_df(
    collab.df_valid, valid_pct=0.0, user_name='Sample ID', item_name='peptide', rating_name='intensity', bs=args.batch_size,
    shuffle=False, device=device)
collab.dl_train.show_batch()

In [ ]:
from fastai.data.core import DataLoaders
collab.dls = DataLoaders(collab.dl_train.train, collab.dl_valid.train)
if args.cuda:
    collab.dls.cuda()

In [ ]:
collab.dl_valid.show_batch()

In [ ]:
len(collab.dls.classes['Sample ID']), len(collab.dls.classes['peptide'])

In [ ]:
len(collab.dls.train), len(collab.dls.valid)  # mini-batches

Alternatively to the hacky version, one could use a factory method, but there the sampling/Splitting methods would need to be implemented (not using [`RandomSplitter`](https://docs.fast.ai/data.transforms.html#RandomSplitter) somehow)

 - [`TabDataLoader`](https://docs.fast.ai/tabular.core.html#TabDataLoader)
 - uses [`TabularPandas`](https://docs.fast.ai/tabular.core.html#TabularPandas)
 
 > Current problem: No custom splitter can be provided

In [ ]:
# valid_idx = [analysis.df_long.index.get_loc(key=key) for key in analysis.indices_valid]
# splitter = IndexSplitter([valid_idx])
# splitter(collab.df.index)

# # replace in CollabDataloaders for getting splits
# # or directly in TabularCollab
# CollabDataLoaders??

In [ ]:
# # drop NAs before?
#
# from fastai.tabular.all import *
# from fastai.tabular.data import TabularDataLoaders
# collab.dls = TabularDataLoaders.from_df(
#     df=analysis.df_long.reset_index(),
#     procs=[Categorify],
#     valid_idx=valid_idx,
#     cat_names=['Sample ID', 'peptide'],
#     y_names=['intensity'],
#     with_cont=False,
#     y_block=TransformBlock(),
#     bs=64)
# collab.dls.show_batch()
# # Problem: this return a second empty df - > would need to adapt model.

A brief check that the values match roughly

In [ ]:
# from numpy.testing import assert_almost_equal
# UPTODECIMAL = 5
# assert_almost_equal(
#     collab.dls.valid_ds['intensity'].values,
#     analysis.df_long.iloc[valid_idx]['intensity'],
#     decimal=UPTODECIMAL
# )
# print(f"Values match up to the {UPTODECIMAL} decimal.")

### Model

In [ ]:
collab.model_args = {}
collab.model_args['n_samples'] = len(collab.dls.classes['Sample ID'])
collab.model_args['n_peptides'] = len(collab.dls.classes['peptide'])
collab.model_args['dim_latent_factors'] = 20
collab.model_args['y_range'] = (
    int(analysis.df_train['intensity'].min()), int(analysis.df_train['intensity'].max())+1)

print("Args:")
pprint(collab.model_args)


model = vaep_model.DotProductBias(**collab.model_args)
learn = Learner(dls=collab.dls, model=model, loss_func=MSELossFlat())
if args.cuda:
    learn.cuda()
learn.summary()

### Training

In [ ]:
learn.fit_one_cycle(5, 5e-3)

### Evaluation

In [ ]:
collab.dls.valid_ds.items

In [ ]:
# import pandas as pd
# dtype = pd.CategoricalDtype(collab.dls.classes['peptide'], ordered=False)
# pd.Categorical.from_codes(codes=collab.dls.valid_ds.items['Sample ID'], dtype=dtype)

In [ ]:
# show False does not return results..
res = learn.show_results(show=True)  # something similar with return

In [ ]:
df_pred = collab.df_valid.copy()
pred, target = learn.get_preds()
df_pred['intensity_pred_collab'] = pd.Series(
    pred.flatten().numpy(), index=collab.dls.valid.items.index)

npt.assert_almost_equal(
    actual=collab.dls.valid.items.intensity.to_numpy(),
    desired=target.numpy().flatten()
)


def cast_object_to_category(df):
    """Object to category dtype."""
    _columns = df.select_dtypes(include='object').columns
    return df.astype({col: 'category' for col in _columns})


df_pred = cast_object_to_category(df_pred)
df_pred.set_index(['Sample ID', 'peptide'], inplace=True)
df_pred

In [ ]:
# # Adapt to get prediction Dataframe
# encodings, pred, target = learn.get_preds(
#     with_input=True)  # per default validation data
# pred_df = pd.DataFrame([{'Sample ID': collab.dls.classes['Sample ID'][obs[0]], 'peptide': collab.dls.classes['peptide']
#                          [obs[1]], 'intensity_pred_collab': pred_intensity.item(), 'intensity': orig_intensity.item() } for obs, pred_intensity, orig_intensity in zip(encodings, pred, target)])
# # pred_df = pred_df.pivot(index='Sample ID', columns='peptide')
# pred_df.sort_values(by=['Sample ID', 'peptide'])

In [ ]:
(abs(target - pred)).sum() / len(target)

## Denoising Autoencoder (DAE)

### Custom Transforms

- [x] Shift standard normalized data around
    - Error metrics won't be directly comparable afterwards

In [ ]:
from fastai.tabular.all import *

In [ ]:
from fastcore.basics import store_attr
from fastcore.imports import noop
from fastai.data.transforms import Normalize, broadcast_vec


class NormalizeShiftedMean(Normalize):
    "Normalize/denorm batch of `TensorImage` with shifted mean and scaled variance."

    def __init__(self, mean=None, std=None, axes=(0, 2, 3),
                 shift_mu=0.5, scale_var=2): store_attr()

    def setups(self, to: Tabular):
        store_attr(but='to', means=dict(getattr(to, 'train', to).conts.mean()),
                   stds=dict(getattr(to, 'train', to).conts.std(ddof=0)+1e-7))
        self.shift_mu = 0.5
        self.scale_var = 2
        return self(to)

    def encodes(self, to: Tabular):
        to.conts = (to.conts-self.means) / self.stds
        to.conts = to.conts / self.scale_var + self.shift_mu
        return to

    def decodes(self, to: Tabular):
        to.conts = (to.conts - self.shift_mu) * self.scale_var
        to.conts = (to.conts*self.stds) + self.means
        return to

    _docs = dict(encodes="Normalize batch with shifted mean and scaled variance",
                 decodes="Normalize batch with shifted mean and scaled variance")


# test with Tabular data somehow
test_data_view = analysis.df.iloc[:100, :10]

# norm_shifted = Normalize.from_stats(mean=test_data_view.mean(), std=test_data_view.std())
# norm_shifted = NormalizeShiftedMean.from_stats(mean=test_data_view.mean(), std=test_data_view.std())

procs = [NormalizeShiftedMean, FillMissing(add_col=True)]
cont_names = list(set(test_data_view))
to = TabularPandas(test_data_view, procs=procs, cat_names=None,
                   cont_names=cont_names, y_names=None, splits=None, y_block=None, do_setup=True)
to.items

FastAi uses singledispatch internally to modify "Normalization" dependent on the type annotations!

  - see `Transform` [docs](https://fastcore.fast.ai/transform.html#Transform) (`Transform` itself is part of fastcore library)
 
  - see the `Transform`'s meta class [`__call__` function](https://github.com/fastai/fastcore/blob/ae8148c85a0c57cc7fd6aa29fa13bdbfbe59be22/fastcore/transform.py#L33-L39) for the "singledispatch"/"typedispatch" functionality shown below 
  - `Normalize` typedispatch added for each application, [here `Tabular`](https://github.com/fastai/fastai/blob/99d38fec7207db9b4209568bebc85ded7e3d3f1b/fastai/tabular/core.py#L269-L283)
  - checkout [`TabularProc`](https://docs.fast.ai/tabular.core.html#TabularProc) and [`InPlaceTransform`](https://fastcore.fast.ai/transform.html#InplaceTransform)

> Check what happens if one removes `to:Tabular`

In [ ]:
# #This would replace the Normalization for Tabular objects
# @Normalize
# def setups(self, to:Tabular):
#     store_attr(but='to', means=dict(getattr(to, 'train', to).conts.mean()),
#                stds=dict(getattr(to, 'train', to).conts.std(ddof=0)+1e-7))
#     self.shift_mu = 0.5
#     self.scale_var = 2
#     return self(to)

# @Normalize
# def encodes(self, to:Tabular):
#     to.conts = (to.conts-self.means) / self.stds
#     to.conts =  to.conts / self.scale_var + self.shift_mu
#     return to

# @Normalize
# def decodes(self, to:Tabular):
#     to.conts = (to.conts - self.shift_mu) * self.scale_var
#     to.conts = (to.conts*self.stds ) + self.means
#     return to

# # test with Tabular data somehow
# test_data_view = analysis.df.iloc[:100, :10]

# # norm_shifted = Normalize.from_stats(mean=test_data_view.mean(), std=test_data_view.std())
# # norm_shifted = NormalizeShiftedMean.from_stats(mean=test_data_view.mean(), std=test_data_view.std())

# procs = [Normalize, FillMissing(add_col=True)]
# cont_names = list(set(test_data_view))
# to = TabularPandas(test_data_view, procs=procs, cat_names=None,
#                    cont_names=cont_names, y_names=None, splits=None, y_block=None, do_setup=True)
# to.items

### DataLoaders

In [ ]:
from fastai.callback.core import Callback

from fastai.data.core import DataLoaders
# from fastai.data.transforms import Normalize

# from fastai.learner import *
from fastai.learner import Learner
from fastai.losses import MSELossFlat


# https://docs.fast.ai/tabular.core.html#FillStrategy
# from fastai.tabular.core import FillMissing
# from fastai.tabular.core import TabularPandas

In [ ]:
# revert format
analysis.df_train = analysis.df_train['intensity'].unstack() #undo using `stack`
analysis.df_valid = analysis.df_valid['intensity'].unstack()

Mean and std. dev. from training data

In [ ]:
# norm = Normalize.from_stats(analysis.df_train.mean(), analysis.df_valid.std()) # copy interface?
NORMALIZER = Normalize # NormalizeShiftedMean

#### Training data

procs passed to TabluarPandas are handled internally 
  1. not necessarily in order
  2. with setup call (using current training data)

In [ ]:
procs = [NORMALIZER, FillMissing(add_col=True)]
cont_names = list(analysis.df_train.columns)

to = TabularPandas(analysis.df_train, procs=procs, cont_names=cont_names)
print("Tabular object:", type(to))

to.items # items reveals data in DataFrame

Better manuelly apply `Transforms` on `Tabluar` type

In [ ]:
cont_names = list(analysis.df_train.columns)
to = TabularPandas(analysis.df_train, cont_names=cont_names, do_setup=False)


tf_norm = NORMALIZER()
_ = tf_norm.setups(to) # returns to
tf_fillna = FillMissing(add_col=True)
_ = tf_fillna.setup(to)

print("Tabular object:", type(to))
# _ = (procs[0]).encodes(to)
to.items # items reveals data in DataFrame

Check mean and standard deviation after normalization

In [ ]:
to.items.iloc[:, :10].describe()  # not perferct anymore as expected

Mask is added as type bool

In [ ]:
to.items.dtypes.value_counts()

with the suffix `_na` where `True` is indicating a missing value replaced by the `FillMissing` transformation

In [ ]:
to.cont_names, to.cat_names

In [ ]:
assert len(to.valid) == 0

#### Validation data

- reuse training data with different mask for evaluation
- target data is the validation data
    - switch between training and evaluation mode for setting comparison

In [ ]:
_df_valid = TabularPandas(analysis.df_valid,cont_names=analysis.df_valid.columns.tolist())
# assert analysis.df_valid.isna().equals(y_valid.items.isna())
_df_valid = tf_norm.encodes(_df_valid)

In [ ]:
_df_valid.items.iloc[:,:10].describe()

In [ ]:
# Validation dataset
# build validation DataFrame with mask according to validation data
# FillNA values in data as before, but do not add categorical columns (as this is done manuelly)
_valid_df = to.conts  # same data for predictions
_valid_df = _valid_df.join(analysis.df_valid.isna(), rsuffix='_na')  # mask
_valid_df = _valid_df.join(_df_valid.items, rsuffix='_val')  # target
_valid_df

In [ ]:
from fastai.tabular.core import TabularPandas
procs = None #[norm, FillMissing(add_col=False)]  # mask is provided explicitly

cont_names = list(analysis.df_train.columns)
cat_names = [f'{s}_na' for s in cont_names]
y_names = [f'{s}_val' for s in cont_names]

splits = None
y_block = None
to_valid = TabularPandas(_valid_df, procs=procs, cat_names=cat_names, cont_names=cont_names,
                         y_names=y_names, splits=splits, y_block=y_block, do_setup=True)
to_valid.items

In [ ]:
stats_valid = to_valid.targ.iloc[:, :100].describe()
stats_valid

In [ ]:
to_valid.cats # True = training data ("fill_na" transform sets mask to true in training data where values are replaced)

In [ ]:
assert list(to_valid.cat_names) == list(
    _valid_df.select_dtypes(include='bool').columns)
assert to_valid.cats.equals(analysis.df_valid.isna().add_suffix('_na'))

### PyTorch DataLoader

- [ ] can they plukked in efficiently as suggested by fastai paper?

In [ ]:
# from vaep.transform import ShiftedStandardScaler

# args_ae = {}
# args_ae['SCALER'] = StandardScaler
# args_ae['SCALER'] = ShiftedStandardScaler

# # select initial data: transformed vs not log transformed
# scaler = args_ae['SCALER'](scale_var=2).fit(analysis.df_train)
# # five examples from validation dataset
# scaler.transform(analysis.df_train).describe(percentiles=[0.025, 0.975])

In [ ]:
# from torchvision import transforms
# from torch.utils.data import DataLoader
# from vaep.io.datasets import PeptideDatasetInMemoryMasked

# # ToDo: replace with helper class (see below)
# tf_norm = None  # replace with Normalizer

# dataset_train = PeptideDatasetInMemoryMasked(
#     data=scaler.transform(analysis.df_train.values))
# dataset_valid = PeptideDatasetInMemoryMasked(
#     data=scaler.transform(analysis.df_valid.values))
# dl_train = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True)
# dl_valid = DataLoader(dataset_valid, batch_size=args.batch_size, shuffle=False)

### Mix and match dataloaders

- train dataloader in both TabularPandas objects used
- train dataloader in dataloaders used in both case

In [ ]:
args.batch_size
dl_train = to.dataloaders(shuffle_train=True, shuffle=False,
                          bs=args.batch_size).train  # , after_batch=after_batch)
dl_valid = to_valid.dataloaders(
    shuffle_train=False, shuffle=False, bs=args.batch_size).train

In [ ]:
dls = DataLoaders(dl_train, dl_valid)
b = dls.train.one_batch()
[x.shape for x in b]  # cat, cont, target

In [ ]:
dls = DataLoaders(dl_train, dl_valid)
b = dls.valid.one_batch()
[x.shape for x in b]  # cat, cont, target

### Model

- standard PyTorch Model from before

In [ ]:
M = analysis.df_train.shape[-1]
model = vaep_model.Autoencoder(n_features=M, n_neurons=int(
    M/2), last_decoder_activation=None, dim_latent=30)

### Callbacks

- controll training loop
    - set what is data
    - what should be used for evaluation (differs for training and evaluation mode)

In [ ]:
class ModelAdapter(Callback):
    """Models forward only expects on input matrix. 
    Apply mask from dataloader to both pred and targets."""

    def __init__(self, p=0.1):
        self.do = torch.nn.Dropout(p=p)  # for denoising AE

    def before_batch(self):
        """Remove cont. values from batch (mask)"""
        # assert self.yb
        mask, data = self.xb  # x_cat, x_cont
        self.learn._mask = mask != 1
        if self.training:
            self.learn.yb = (data[self.learn._mask],)
        # dropout data using median
        self.learn.xb = (self.do(data),)
        #         # could be function handeled by switch set at beginning of evaluation/training?
        #         if self.training:
        #             self.learn.yb = (data[self.learn._mask],)
        #         else:
        #             # self.y is not available at "before_batch" - > why?
        #             self.learn.yb = (self.y[mask],)

    def after_pred(self):
        M = self._mask.shape[-1]

        if not self.training:
            if len(self.yb):
                self.learn.yb = (self.y[self.learn._mask],)
                self.val_targets.append(self.learn.yb[0])
                

        #         self.learn.pred = self.pred.view(-1, M)[self._mask] #is this flat?
        self.learn.pred = self.pred[self._mask] #is this flat?
        if not self.training:
            self.val_preds.append(self.learn.pred)
        
    def before_validate(self):
        "containers for current predictions"
        self.learn.val_preds,self.learn.val_targets = [],[]


### Learner: Fastai Training Loop

In [ ]:
learn = Learner(dls=dls, model=model,
                loss_func=MSELossFlat(), cbs=ModelAdapter())

In [ ]:
learn.show_training_loop()

In [ ]:
learn.summary()

In [ ]:
suggested_lr = learn.lr_find()
suggested_lr

#### Train one batch

In [ ]:
b = learn.dls.one_batch()
learn.one_batch(0, b)

In [ ]:
b

#### Train epochs

In [ ]:
learn.fit_one_cycle(30, lr_max=suggested_lr.lr_min)

In [ ]:
learn.val_preds, learn.val_targets #

In [ ]:
learn.recorder.plot_loss()

In [ ]:
# L(zip(learn.recorder.iters, learn.recorder.values))


### Evaluation

In [ ]:
pred, target = learn.get_preds(act=noop, concat_dim=0, reorder=False) # reorder True: Only 500 predictions returned
len(pred), len(target)  

MSE on transformed data is not too interesting for comparision between models if these use different standardizations

In [ ]:
learn.loss_func(pred, target) # MSE in transformed space not too interesting

In [ ]:
# check target is in expected order
Y = dls.valid.targ

npt.assert_almost_equal(
    actual=target.numpy(),
    desired=Y.stack().to_numpy()
)

In [ ]:
def transform_preds(pred:torch.Tensor, index:pd.Index):
    pred = pd.Series(pred, index).unstack()
    pred = TabularPandas(pred, cont_names=list(pred.columns))
    _ = tf_norm.decode(pred)
    pred = pred.items.stack()
    return pred

df_pred['intensity_pred_dae'] = transform_preds(pred=pred, index=analysis.df_valid.stack().index)
df_pred

## Variational Autoencoder (VAE)

### Transforms - FastAi MinMaxScaler

In [ ]:
# X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
# X_scaled = X_std * (max - min) + min

### Scikit Learn MinMaxScaler

- [docs](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [ ]:
from vaep.transform import MinMaxScaler

args_vae = {}
args_vae['SCALER'] = MinMaxScaler
# select initial data: transformed vs not log transformed
scaler = args_vae['SCALER']().fit(analysis.df_train)
scaler.transform(analysis.df_valid.iloc[:5])

## Compare the 3 models

In [ ]:
import sklearn.metrics as sklm
pred_columns = df_pred.columns[df_pred.columns.str.contains('pred')]
scoring =     [('MSE',sklm.mean_squared_error),
    ('MAE',sklm.mean_absolute_error)]

y_true = df_pred['intensity']

metrics = {}
for col in pred_columns:
    metrics[col] = dict(
        [(k, f(y_true=y_true, y_pred=df_pred[col])) for k, f in scoring]
    )
    
#     sklm.mean_absolute_error

pd.DataFrame(metrics)

Save final prediction values of validation data for later comparison.

In [ ]:
df_pred.to_csv(config.FOLDER_DATA / f"{config.FOLDER_DATA}_valid_pred")